# Cálculo del Índice Estandarizado de Precipitación (SPI)

**Este notebook pertenece al Trabajo Fin de Grado del Curso de Adaptación al Grado de Ingeniería en Informática**


*Fecha:* Mayo 2022

*Autor:* Ruth Lospitao Ruiz


*Email:* rlospitao@gmail.com

# Introducción

El Índice Estandarizado de Precipitación (SPI) es  la diferencia de la precipitación media para un período de tiempo específico, dividida por la desviación estándar; donde la media y la desviación estándar se determinan a partir de registros anteriores.

Este libro, se encarga de abrir un fichero csv con las predicciones de precipitaciones, para calcular el índice SPI y poder identificaR si existirá sequía y su categoría.

<table>
 <tr><td>Valores SPI</td><td>Categoría</td></tr>
  <tr><td>0 a -0.99</td><td>Sequía leve</td></tr>
  <tr><td>-1.00 a -1.49</td><td>Sequía leve</td></tr>

  <tr><td>-1.50 a -1.99</td><td>Sequía leve</td></tr>  
  <tr><td>-<=-2.00</td><td>Sequía leve</td></tr>  
</table>

# Importación de paquetes y definición de funciones

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


In [2]:
#Standardized Precipitation Index Function
def spi(ds, thresh):
    #ds - data ; thresh - time interval / scale
    
    #Rolling Mean / Moving Averages
    ds_ma = ds.rolling(thresh, center=False).mean()
    
    #Natural log of moving averages
    ds_In = np.log(ds_ma)
    ds_In[ np.isinf(ds_In) == True] = np.nan  #Change infinity to NaN
    
    #Overall Mean of Moving Averages
    ds_mu = np.nanmean(ds_ma)
    
    #Summation of Natural log of moving averages
    ds_sum = np.nansum(ds_In)
        
    #Computing essentials for gamma distribution
    n = len(ds_In[thresh-1:])                  #size of data
    A = np.log(ds_mu) - (ds_sum/n)             #Computing A
    alpha = (1/(4*A))*(1+(1+((4*A)/3))**0.5)   #Computing alpha  (a)
    beta = ds_mu/alpha                         #Computing beta (scale)
    
    #Gamma Distribution (CDF)
    gamma = st.gamma.cdf(ds_ma, a=alpha, scale=beta)  
    
    #Standardized Precipitation Index   (Inverse of CDF)
    norm_spi = st.norm.ppf(gamma, loc=0, scale=1)  #loc is mean and scale is standard dev.
    
    return ds_ma, ds_In, ds_mu, ds_sum, n, A, alpha, beta, gamma, norm_spi

# Carga de datos

In [4]:
df = pd.read_excel("data/rainfall_3195.xlsx", sheet_name = 0, usecols=["precipitation"])  # se lee un fichereo de ejemplo que tendría las predicciones
df.head(5)

,precipitation
0,143.5
1,98.6
2,121.7
3,111.0
4,108.0


In [5]:
df = df.set_index(
    pd.date_range('1985','2021', freq='M')
)

In [6]:
df.head(5)

,precipitation
1985-01-31,143.5
1985-02-28,98.6
1985-03-31,121.7
1985-04-30,111.0
1985-05-31,108.0


# Estimación del índice SPI

In [7]:
spi(df["precipitation"], 12)[9] #[9] tiene el valor norm_spi

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1997: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1997: RuntimeWarning: invalid value encountered in less
  cond1 = (0 < q) & (q < 1)


array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan, -1.15976075, -1.3652348 , -1.18616854, -1.46162183,
       -1.38906773, -1.76636712, -1.88599492, -1.8739126 , -1.81390246,
       -1.55148646, -0.52595333, -0.60599481, -0.85786178, -0.53169094,
       -0.35586895, -0.44278313, -0.48176106, -0.37636489, -0.15087706,
        0.09129107,  0.09476058, -0.00660903, -0.17976071, -0.00837066,
        0.57706586,  0.78128349,  0.27675957,  0.26324746,  0.58996724,
        0.92064506,  1.48360044,  1.44767068,  1.40955572,  1.29314325,
        1.09791224,  1.10929642,  0.29630373, -0.47948379, -0.16566366,
        0.01519571, -0.17433509, -0.32617535, -1.04530096, -1.17976853,
       -1.0328126 , -0.91995841, -1.32835855, -0.03662192,  0.90710913,
        1.06666328,  0.72652486,  0.58190671,  0.39317997,  0.06137738,
       -0.11029909, -0.00766595, -0.0981397 ,  0.00359905,  0.38

In [8]:
times = [3, 6, 9, 12, 24]
for i in times:
    x = spi(df['precipitation'], i)
    df['spi_'+str(i)] = x[9]

In [9]:
df

,precipitation,spi_3,spi_6,spi_9,spi_12,spi_24
1985-01-31,143.5,NaN,NaN,NaN,NaN,NaN
1985-02-28,98.6,NaN,NaN,NaN,NaN,NaN
1985-03-31,121.7,0.446876,NaN,NaN,NaN,NaN
1985-04-30,111.0,0.297066,NaN,NaN,NaN,NaN
1985-05-31,108.0,0.341361,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-08-31,49.6,-1.262952,-0.026982,0.275883,0.811001,-0.124482
2020-09-30,144.3,-0.436296,-0.218340,-0.027053,1.034371,0.168491
2020-10-31,236.3,0.727552,-0.004638,0.233391,1.425055,0.298474
2020-11-30,166.7,1.160700,0.220202,0.749336,0.991593,0.007766


In [10]:
df.to_excel("data/spi_3195.xlsx")